# CRQ1

#### Does the 'fare per mile' change among boroughs? aka, which boroughs charge the passengers with more money per mile?
#### We need to consider the traffic congestion too. The price might depend on the trafic taxis find in their way. 

In [89]:
import numpy as np
import pandas as pd
from datetime import datetime
import time

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(r'C:\Users\samin\Desktop\Homework2\Data\yellow_tripdata_2018-01.csv', usecols =['tpep_pickup_datetime','trip_distance','PULocationID','fare_amount'])

In [ ]:
df.head()

In [ ]:
df.info(memory_usage = 'deep')

In [ ]:
#Change data type to reduce memory usage
df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'])

df['trip_distance']=df['trip_distance'].astype(np.float16)
df['PULocationID']=df['PULocationID'].astype(np.uint16)
df['fare_amount']=df['fare_amount'].astype(np.float16)

In [ ]:
df.info(memory_usage='deep')

In [ ]:
#We need to extract Month from out datetime object
df['Month'] = pd.DatetimeIndex(df['tpep_pickup_datetime']).month

In [ ]:
df.head()

In [ ]:
df.drop(columns=['tpep_pickup_datetime'], inplace=True)

In [ ]:
df.info(memory_usage ='deep')

In [ ]:
df.head()

In [ ]:
df.groupby('Month').count()

In [ ]:
#delete those rows with month!=1
df.drop(df[(df.Month == 2) | (df.Month == 7) | (df.Month == 12)].index, inplace=True)

In [ ]:
df.info(memory_usage ='deep')

In [ ]:
df['Month']=df['Month'].astype(np.uint8)

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
#clean based on two other columns: 'trip_distance' and  'fare_amount'
#'fare_amount' shouldn't be zero also 'trip_distance'. Negative values also make the dataset dirty so we get rid of them. 

df.drop(df[(df.trip_distance==0)].index, inplace=True)
df.drop(df[(df.fare_amount==0)].index, inplace=True)
df = df[~(df['trip_distance'] < 0)]
df = df[~(df['fare_amount'] < 0)]

In [ ]:
df.shape

### Part1: Find the fare amount per mile. For each borough, calculate mean and standard deviation of values. Plot the result and run the t-test on each pair of two boroughs possible

In [ ]:
df['price_per_mile'] = df['fare_amount']//df['trip_distance']

In [ ]:
df['price_per_mile'].min()

In [ ]:
df.head(10)

In [ ]:
df.drop(columns = ['trip_distance', 'fare_amount'], inplace=True)

In [ ]:
df.info(memory_usage='deep')

In [ ]:
df.head()

In [ ]:
#Read 'taxi_zone_lookup' dataset

borough = pd.read_csv(r'C:\Users\samin\Desktop\Homework2\taxi_zone_lookup.csv', usecols = ['LocationID', 'Borough'])

In [ ]:
borough.info(memory_usage = 'deep')

In [ ]:
borough.head()

In [ ]:
#Merge df and borough 

result = pd.merge(df,
                 borough,
                 left_on='PULocationID',
                 right_on='LocationID',
                 how='inner' 
                 )


In [ ]:
result.info(memory_usage='deep')

In [ ]:
result.head()

In [ ]:
del df
del borough

In [ ]:
result.drop(columns = ['PULocationID', 'LocationID'], inplace =  True)

In [ ]:
result.info(memory_usage = 'deep')

In [ ]:
result.head()

In [ ]:
result['price_per_mile'].min()

In [ ]:
result['price_per_mile'].max()

In [ ]:
sns.distplot(result['price_per_mile'], kde=False, bins=15, hist_kws=dict(edgecolor="k", linewidth=2))

In [ ]:
#result.drop([])
result.drop(result[(result.price_per_mile <1) | (result.price_per_mile>200)].index, inplace=True)

In [ ]:
sns.distplot(result['price_per_mile'], kde=False, bins=15, hist_kws=dict(edgecolor="k", linewidth=2))

In [ ]:
result['price_per_mile'].min()

In [ ]:
#To have a normalized distribution, we drop prices higher than 25
result.drop(result[(result.price_per_mile >15)].index, inplace=True)

In [ ]:
sns.distplot(result['price_per_mile'], kde=False, bins=15, hist_kws=dict(edgecolor="k", linewidth=2))

In [ ]:
#looks almost narmalized

In [ ]:
result.groupby('Borough').count()

In [ ]:
#delete rows with Borough='unknown' and Borough='EWR'
result.drop(result[(result.Borough == 'EWR') | (result.Borough == 'Unknown')].index, inplace=True)
result.shape

In [ ]:
result.info(memory_usage='deep')

In [ ]:
M=result.groupby('Borough')['price_per_mile'].mean()
print(M)

In [ ]:
#plotting
M.plot.bar()

#### Manhattan seems to be the most expensive area, Staten Island comes in the second palce. Queens charges each passenger the lowest amount

In [ ]:
S=result.groupby('Borough')['price_per_mile'].std()

In [ ]:
print(S)

In [ ]:
S.plot.bar()

#### Seems to be that the variation in values for Staten Island is the highest. This makes us wonder if the its mean has a true value. 

# T-Test

In [ ]:
from scipy import stats
#import scipy.stats 
#Welch's t-test
#scipy.stats.ttest_ind(two_data.loc['cat'], two_data.loc['cat2'], equal_var=False)#default is two sided t-test 
#(ttest_ind= independent t- test also called two sapmle t-test or student's t-test)


In [ ]:
#independent t-test assumption


In [91]:
#ttest_ind(bronx, brooklyn)

#which type of t-test to apply?
#We check if independent t-test is going to have reliable results for our case?
boroughs = ['Bronx', 'Brooklyn', 'Manhattan', 'Staten Island', 'Queens']
iterables = [['Bronx', 'Brooklyn', 'Manhattan', 'Staten Island', 'Queens'], ['t-value', 'p-value']]
my_index = pd.MultiIndex.from_product(iterables)
dt = pd.DataFrame(index=my_index, columns=boroughs)


In [90]:

for i in boroughs:
    a = result.loc[result.Borough==i]["price_per_mile"]
    for j in boroughs:
            b = result.loc[result.Borough==j]["price_per_mile"]
            b = b.dropna()
            t2, p2 = stats.ttest_ind(a,b)
            #T.append(t2)
            #P.append(p2)
            dt.loc[(i,"t-value"),j]=t2
            dt.loc[(i,"p-value"),j]=p2
            #if(p2>0.05):
            #    dt.loc[(i,"H0 hypothesis"),j]='Fail to Reject H0'
            #else:
            #    dt.loc[(i,"H0 hypothesis"),j]='Reject H0'

In [88]:
dt

Bronx Brooklyn Manhattan Staten Island Queens
Bronx         t-value           0        0        -0      -2.98818      0
              p-value           1        1         1    0.00281693      1
Brooklyn      t-value          -0        0        -0            -0      0
              p-value           1        1         1             1      1
Manhattan     t-value           0        0         0             0      0
              p-value           1        1         1             1      1
Staten Island t-value     2.98818        0        -0             0      0
              p-value  0.00281693        1         1             1      1
Queens        t-value          -0       -0        -0            -0      0
              p-value           1        1         1             1      1

##### these are the t-values and p-values that I get! I am reading to understand if this t-test is appropriate for our data, hence the results are meaningful 
